# NWB File Examples

This example introduces the NWB file format, creating an example datafile with dummy data.

This example is adapted from: https://pynwb.readthedocs.io/en/stable/tutorials/general/file.html

This example uses [pynwb](https://github.com/NeurodataWithoutBorders/pynwb), the Python tool for interacting with NWB files.

In [1]:
%config Completer.use_jedi = False

In [2]:
import os
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np

# Import utilities from pynwb
from pynwb import NWBFile, NWBHDF5IO, TimeSeries

## Settings

In [3]:
# Set path for data files
data_path = 'data'

## Initialize an NWB File

NWB files are a file specification for [HDF5 files](https://www.hdfgroup.org/solutions/hdf5/).

HDF5 files are a file format defined for large and/or heterogenous data. 

To get started initializing a HDF5 file, we can `NWBFile` function. 

In [4]:
# Get local timezone
timezone = tzlocal() 

# Set time zone information
start_time = datetime(2021, 3, 20, tzinfo=tzlocal())
create_date = datetime(2021, 3, 20, tzinfo=tzlocal())

In [5]:
# Define some metadata for the file
session_info = 'NWBFile test'
session_id = 'NWB123'

In [6]:
# Initialize an NWB file
nwbfile = NWBFile(session_description=session_info,
                  identifier=session_id,
                  session_start_time=start_time,
                  file_create_date=create_date)

## Add Data to NWB File

In [7]:
# Create some dummy test data
data = np.arange(100, 200, 10)
timestamps = np.arange(10.)

In [8]:
# Create dummy time series object
test_ts = TimeSeries(name='test_timeseries', data=data, unit='m', timestamps=timestamps)

In [9]:
# Add the dummy time series as an acquisition element
nwbfile.add_acquisition(test_ts)

In [10]:
# Check the acquisition element
nwbfile.acquisition['test_timeseries']

test_timeseries pynwb.base.TimeSeries at 0x140632470617248
Fields:
  comments: no comments
  conversion: 1.0
  data: [100 110 120 130 140 150 160 170 180 190]
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
  timestamps_unit: seconds
  unit: m

In [11]:
# Get the newly added acquisition
test_acq = nwbfile.get_acquisition('test_timeseries')

## Save out NWB File

In [12]:
# Save out an example NWB file
with NWBHDF5IO('data/nwb_basics.nwb', 'w') as io:
    io.write(nwbfile)

In [13]:
# Check for saved out NWB files
nwb_files = [file for file in os.listdir('.') if file.split('.')[-1] == 'nwb']
print(nwb_files)

[]


## Validating NWB Files

NWB has utilities to validate NWB files.

More information: https://pynwb.readthedocs.io/en/stable/validation.html

In [14]:
# Run command line tool to validate NWB file
!python -m pynwb.validate nwb_basics.nwb

The file nwb_basics.nwb does not exist.


## Read in NWB File

In [15]:
# Read in an example NWB file
io = NWBHDF5IO('data/nwb_basics.nwb', 'r')
nwbfile_in = io.read()

In [16]:
# Check the overview information of our example file
nwbfile_in

root pynwb.file.NWBFile at 0x140632506119984
Fields:
  acquisition: {
    test_timeseries <class 'pynwb.base.TimeSeries'>
  }
  file_create_date: [datetime.datetime(2021, 3, 20, 0, 0, tzinfo=tzoffset(None, -14400))]
  identifier: NWB123
  session_description: NWBFile test
  session_start_time: 2021-03-20 00:00:00-04:00
  timestamps_reference_time: 2021-03-20 00:00:00-04:00

In [17]:
# Extract an example data object
test_ts_in = nwbfile_in.acquisition['test_timeseries']

In [18]:
# Check the example data object
print(test_ts_in)

test_timeseries pynwb.base.TimeSeries at 0x140632512249040
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (10,), type "<i8">
  description: no description
  interval: 1
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (10,), type "<f8">
  timestamps_unit: seconds
  unit: m



In [19]:
# Close the file reader
io.close()